## 🧭 Introducción

En esta práctica trabajaremos con el dataset real **Titanic**, muy utilizado en ciencia de datos por su riqueza y variedad de variables. A diferencia de los datasets anteriores, este contiene:

- **Campos vacíos (valores faltantes)** en varias columnas.
- **Variables categóricas**, como nombres, género, clases del pasaje, entre otras.
- Mezcla de **datos numéricos y no numéricos**, lo que requiere un preprocesamiento más cuidadoso.

El objetivo de esta actividad es practicar conceptos fundamentales del tratamiento de datos reales antes del entrenamiento de modelos:

- 🔍 Identificar y tratar **valores faltantes**
- 🔡 Codificar **variables categóricas** (label encoding, one-hot encoding)
- 🧹 Preparar un conjunto de datos limpio, numérico y listo para aplicar modelos de machine learning

Este paso es clave en cualquier proyecto de ciencia de datos y nos permite adaptar los datos a las exigencias de los algoritmos de aprendizaje automático.

---


# 🔹 Paso 1: Carga del dataset Titanic
📌 El dataset "Titanic" contiene información sobre los pasajeros del famoso barco. Es útil para tareas de clasificación: por ejemplo, predecir quién sobrevivió (survived).

In [16]:
# 🚢 Cargar el dataset Titanic desde seaborn y explorar su contenido
import seaborn as sns
import pandas as pd

# Cargar el dataset como un DataFrame
df = sns.load_dataset("titanic")

# Mostrar las primeras 5 filas
print("Primeras filas: ",  "\n", df.head(),  "\n") # El '\n' representa un salto de línea

# Mostrar la forma del dataset (filas, columnas)
print("Filas y columnas: ", df.shape,  "\n")



Primeras filas:  
    survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True   

Filas y columnas:  (891, 15) 



In [15]:
# Información general de columnas, tipos y valores faltantes
print("Información general:", "\n")
print( df.info(),  "\n")

# Estadísticas descriptivas de columnas numéricas
print("Estadísticas descriptivas de columnas numéricas: ", "\n", df.describe())

Información general: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None 

Estadísticas descriptivas de columnas numéricas: 

---

# **Análisis de la Información General del DataFrame**

El resultado de la función `df.info()` nos proporciona una visión general crucial de nuestro conjunto de datos, y en este caso, revela aspectos importantes que no habíamos encontrado en nuestros ejemplos anteriores con datos puramente numéricos y completos.

En primer lugar, vemos que tenemos un DataFrame con **891 entradas (filas)**, indexadas desde 0 hasta 890. Esto representa el número total de observaciones en nuestro conjunto de datos.

A continuación, se detalla la información de cada una de las **15 columnas**. Para cada columna, observamos dos aspectos fundamentales:

* **`Non-Null Count` (Conteo de Valores No Nulos):** Esta columna nos indica cuántos valores **no faltantes** hay en cada una de las columnas. Aquí es donde encontramos una novedad importante: **algunas columnas tienen menos de 891 valores no nulos**. Específicamente, notamos que la columna `age` tiene solo 714 valores no nulos, y la columna `embarked` y `embark_town` tienen 889. La columna más llamativa es `deck`, con tan solo 203 valores no nulos. **Esto significa que tenemos datos faltantes en estas columnas.** En un contexto de Machine Learning, los datos faltantes son un desafío que debemos abordar, ya que muchos algoritmos no pueden trabajar directamente con ellos. Necesitaremos aplicar técnicas de **imputación** (reemplazar los valores faltantes con algún valor estimado) o considerar eliminar las columnas o filas con demasiados faltantes, dependiendo del contexto y del impacto en nuestro modelo.

* **`Dtype` (Tipo de Dato):** Esta columna nos muestra el tipo de dato que contiene cada columna. Aquí también encontramos novedades importantes con respecto a nuestros conjuntos de datos anteriores:
    * **`object`:** Varias columnas como `sex`, `embarked`, `who`, `embark_town`, y `alive` son de tipo `object`. Esto generalmente indica que contienen **cadenas de texto (strings)**. Los algoritmos de Machine Learning, en su mayoría, trabajan con datos numéricos. Por lo tanto, necesitaremos aplicar técnicas de **codificación (encoding)** para convertir estas variables categóricas en representaciones numéricas que los modelos puedan entender.
    * **`category`:** Las columnas `class` y `deck` son de tipo `category`. Este es un tipo de dato eficiente para variables categóricas con un número limitado de valores distintos. Internamente, Pandas representa estas categorías con números, lo que puede ser beneficioso para la memoria y la velocidad. Sin embargo, al igual que con las columnas `object`, es posible que necesitemos considerar cómo se interpretan estas categorías en nuestros modelos de ML.
    * **`bool`:** Las columnas `adult_male` y `alone` son de tipo booleano, representando valores True o False. Muchos algoritmos pueden trabajar directamente con datos booleanos, interpretándolos como 0 y 1.
    * **`int64` y `float64`:** Estas son columnas numéricas, como las que ya hemos trabajado (`survived`, `pclass`, `sibsp`, `parch`, `fare`, `age`). `int64` representa números enteros, mientras que `float64` representa números de punto flotante.

Finalmente, la sección de **`memory usage`** nos indica la cantidad de memoria que está utilizando este DataFrame. Esto puede ser importante cuando trabajamos con conjuntos de datos muy grandes.

El último `None` que aparece es el valor de retorno de la función `df.info()`, que en Python es `None` ya que su propósito principal es imprimir la información, no devolver un objeto.

**En resumen, al enfrentarnos a este nuevo conjunto de datos, debemos prestar especial atención a:**

1.  **Los valores faltantes:** Identificar las columnas afectadas y decidir la estrategia para manejarlos.
2.  **Los tipos de datos no numéricos (`object` y `category`):** Planificar cómo vamos a codificar estas variables para que puedan ser utilizadas por nuestros modelos de Machine Learning.

Estos son pasos cruciales en el preprocesamiento de datos para construir modelos efectivos, especialmente cuando trabajamos con datos del mundo real que rara vez son perfectos y puramente numéricos.

---


---

# **🔹 Paso 2: Un Primer Modelo Simplificado - Eliminación de Datos No Numéricos y Faltantes**

En este segundo paso, vamos a realizar un ciclo completo de Machine Learning: división del dataset, entrenamiento de un modelo, realización de predicciones y evaluación de su rendimiento. Sin embargo, para simplificar este primer ejercicio y enfocarnos en el flujo general, vamos a tomar un atajo para lidiar con las columnas que no son numéricas y las filas que contienen valores faltantes.

**Importante:** Esta es una estrategia simplificada y **no es la mejor práctica** en la mayoría de los escenarios reales. Eliminar columnas enteras con información potencialmente útil y filas con datos faltantes puede llevar a una pérdida significativa de información y a un modelo subóptimo. En pasos posteriores, exploraremos técnicas más sofisticadas para manejar estos problemas.

En este paso, nuestro objetivo principal es tener un primer modelo funcional, aunque sea basado en una versión reducida de nuestros datos. Para ello, realizaremos las siguientes acciones:

1.  **Identificar y eliminar las columnas no numéricas:** Seleccionaremos solo las columnas con tipos de datos numéricos (enteros y flotantes) para poder alimentar directamente nuestro modelo.
2.  **Eliminar las filas con valores faltantes (NaN):** Removeremos cualquier fila que contenga al menos un valor faltante en cualquiera de las columnas restantes.

Una vez que tengamos un dataset limpio (aunque reducido), procederemos con los pasos habituales:

1.  **Dividir el dataset** en un conjunto de entrenamiento y un conjunto de prueba.
2.  **Seleccionar y entrenar un modelo de clasificación.** Para este primer ejemplo, utilizaremos un modelo simple como la Regresión Logística.
3.  **Realizar predicciones** sobre el conjunto de prueba.
4.  **Evaluar el rendimiento** del modelo utilizando una métrica adecuada para la clasificación, como la exactitud (accuracy).

Este proceso nos dará una primera idea del flujo de trabajo completo en un problema de clasificación, aunque con las limitaciones impuestas por nuestra estrategia simplificada de manejo de datos.

---

Aquí tienes el código en Google Colab para llevar a cabo estos pasos:

In [19]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Asumiendo que 'df' es tu DataFrame cargado en el Paso 1

# 1. Seleccionar solo las columnas numéricas
df_numeric = df.select_dtypes(include=['int64', 'float64'])
print("DataFrame solo con columnas numéricas:")
print(df_numeric.head())
print("\nForma del DataFrame numérico:", df_numeric.shape)

# 2. Eliminar las filas con valores faltantes
df_cleaned = df_numeric.dropna()
print("\nDataFrame después de eliminar filas con NaN:")
print(df_cleaned.head())
print("\nForma del DataFrame limpio:", df_cleaned.shape)

# 3. Definir las variables predictoras (X) y la variable objetivo (y)
X = df_cleaned.drop('survived', axis=1)
y = df_cleaned['survived']

# 4. Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nForma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

# 5. Inicializar y entrenar un modelo de Regresión Logística
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train, y_train)

# 6. Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# 7. Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("\nExactitud del modelo en el conjunto de prueba:", accuracy)

DataFrame solo con columnas numéricas:
   survived  pclass   age  sibsp  parch     fare
0         0       3  22.0      1      0   7.2500
1         1       1  38.0      1      0  71.2833
2         1       3  26.0      0      0   7.9250
3         1       1  35.0      1      0  53.1000
4         0       3  35.0      0      0   8.0500

Forma del DataFrame numérico: (891, 6)

DataFrame después de eliminar filas con NaN:
   survived  pclass   age  sibsp  parch     fare
0         0       3  22.0      1      0   7.2500
1         1       1  38.0      1      0  71.2833
2         1       3  26.0      0      0   7.9250
3         1       1  35.0      1      0  53.1000
4         0       3  35.0      0      0   8.0500

Forma del DataFrame limpio: (714, 6)

Forma de X_train: (571, 5)
Forma de X_test: (143, 5)
Forma de y_train: (571,)
Forma de y_test: (143,)

Exactitud del modelo en el conjunto de prueba: 0.7062937062937062



# Explicación del Código Paso 2:

* **`df.select_dtypes(include=['int64', 'float64'])`:** Selecciona del DataFrame original (`df`) solo las columnas cuyos tipos de datos son enteros de 64 bits (`int64`) o números de punto flotante de 64 bits (`float64`).
* **`df_numeric.dropna()`:** Crea un nuevo DataFrame (`df_cleaned`) eliminando todas las filas que contengan al menos un valor `NaN` (Not a Number) en cualquiera de las columnas.
* **Definición de `X` e `y`:** Asumimos que la columna `'survived'` es nuestra variable objetivo (la que queremos predecir). Separamos las características predictoras (`X`) eliminando la columna objetivo y la variable objetivo (`y`). **Si tu objetivo es diferente, asegúrate de ajustar el nombre de la columna.**
* **`train_test_split()`:** Divide nuestros datos limpios en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%). `random_state` se utiliza para asegurar la reproducibilidad de la división.
* **`LogisticRegression()`:** Inicializamos un modelo de Regresión Logística. El argumento `solver='liblinear'` es una buena opción para datasets pequeños o medianos.
* **`model.fit(X_train, y_train)`:** Entrenamos el modelo utilizando el conjunto de entrenamiento. El modelo aprende la relación entre las características en `X_train` y la variable objetivo en `y_train`.
* **`model.predict(X_test)`:** Utilizamos el modelo entrenado para hacer predicciones sobre el conjunto de prueba (`X_test`).
* **`accuracy_score(y_test, y_pred)`:** Comparamos las predicciones del modelo (`y_pred`) con los valores reales de la variable objetivo en el conjunto de prueba (`y_test`) para calcular la exactitud, que es el porcentaje de predicciones correctas.

---

# **🔹 Paso 3: Imputación y Codificación para un Mejor Preprocesamiento**
*En este Paso, vamos a abordar de manera más adecuada los desafíos de los datos no numéricos y los valores faltantes. En lugar de simplemente eliminarlos, aplicaremos técnicas de **imputación** para rellenar los valores faltantes y de **codificación** para convertir las variables categóricas en un formato numérico que nuestros modelos puedan entender.*

En el paso anterior, realizamos un ciclo completo de Machine Learning de forma simplificada, eliminando información valiosa al descartar columnas no numéricas y filas con datos faltantes. En este Paso 3, vamos a mejorar significativamente nuestro preprocesamiento de datos aplicando técnicas más sofisticadas:

1.  **Imputación de Valores Faltantes:** Para las columnas que contienen valores `NaN`, utilizaremos estrategias para estimar y reemplazar estos valores faltantes. La elección de la estrategia de imputación dependerá del tipo de dato de la columna y de la naturaleza de los datos. Algunas estrategias comunes incluyen:
    * Para columnas numéricas: Imputar con la media, la mediana o un valor constante.
    * Para columnas categóricas: Imputar con la moda (el valor más frecuente) o una nueva categoría como "Desconocido".

2.  **Codificación de Variables Categóricas:** Las columnas con tipos de datos `object` o `category` contienen información categórica que la mayoría de los algoritmos de Machine Learning no pueden procesar directamente. Necesitamos convertirlas a un formato numérico. Algunas técnicas comunes de codificación son:
    * **One-Hot Encoding:** Crea nuevas columnas binarias (0 o 1) para cada categoría única en la variable original. Es útil cuando las categorías no tienen un orden inherente.
    * **Label Encoding:** Asigna un número entero único a cada categoría. Es apropiado para variables ordinales (donde las categorías tienen un orden).

Nuestro objetivo en este paso es preparar un dataset más completo y representativo para el entrenamiento de modelos, reteniendo la información valiosa que descartamos en el paso anterior. Luego, como hicimos antes, dividiremos el dataset preprocesado, entrenaremos un modelo y lo evaluaremos.

---

Aquí tienes el código en Google Colab para realizar la imputación y la codificación. Vamos a aplicar estrategias comunes, pero recuerda que la elección de la mejor estrategia a menudo requiere un análisis más profundo de los datos.


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Asumiendo que 'df' es tu DataFrame cargado en el Paso 1

# 1. Separar la variable objetivo
X = df.drop('survived', axis=1)
y = df['survived']

# 2. Identificar columnas numéricas y categóricas
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns

# 3. Imputar valores faltantes
# Imputación numérica con la media
numerical_imputer = SimpleImputer(strategy='mean')
X[numerical_cols] = numerical_imputer.fit_transform(X[numerical_cols])

# Imputación categórica con la moda
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = categorical_imputer.fit_transform(X[categorical_cols])

print("DataFrame después de la imputación:")
print(X.head())
print("\nValores nulos después de la imputación:")
print(X.isnull().sum())

# 4. Codificar variables categóricas usando One-Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse=False para obtener un array NumPy
X_encoded = encoder.fit_transform(X[categorical_cols])
feature_names = encoder.get_feature_names_out(categorical_cols)
X_encoded_df = pd.DataFrame(X_encoded, index=X.index, columns=feature_names)

# Concatenar las columnas numéricas con las columnas codificadas
X_processed = pd.concat([X[numerical_cols], X_encoded_df], axis=1)

print("\nDataFrame después del One-Hot Encoding:")
print(X_processed.head())
print("\nForma del DataFrame preprocesado:", X_processed.shape)

# 5. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("\nForma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

# 6. Inicializar y entrenar un modelo de Regresión Logística
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# 7. Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# 8. Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("\nExactitud del modelo en el conjunto de prueba (con imputación y codificación simplificadas):", accuracy)

DataFrame después de la imputación:
   pclass     sex   age  sibsp  parch     fare embarked  class    who  \
0     3.0    male  22.0    1.0    0.0   7.2500        S  Third    man   
1     1.0  female  38.0    1.0    0.0  71.2833        C  First  woman   
2     3.0  female  26.0    0.0    0.0   7.9250        S  Third  woman   
3     1.0  female  35.0    1.0    0.0  53.1000        S  First  woman   
4     3.0    male  35.0    0.0    0.0   8.0500        S  Third    man   

  adult_male deck  embark_town alive  alone  
0       True    C  Southampton    no  False  
1      False    C    Cherbourg   yes  False  
2      False    C  Southampton   yes   True  
3      False    C  Southampton   yes  False  
4       True    C  Southampton    no   True  

Valores nulos después de la imputación:
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
aliv


---

# Explicación del código Paso 3

Este código representa un flujo de trabajo fundamental en Machine Learning, que incluye la preparación de los datos (preprocesamiento) y el entrenamiento de un modelo de clasificación. Vamos a desglosar cada sección:

**1. Importación de Librerías Necesarias:**

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
```

* **`import pandas as pd`**: Importa la librería Pandas, que es esencial para trabajar con estructuras de datos tabulares como los DataFrames. Usaremos `pd` como un alias para referirnos a ella de forma más concisa.
* **`from sklearn.model_selection import train_test_split`**: Importa la función `train_test_split` de la librería Scikit-learn (`sklearn`). Esta función nos permitirá dividir nuestro conjunto de datos en dos partes: un conjunto para entrenar nuestro modelo y otro para evaluar su rendimiento.
* **`from sklearn.linear_model import LogisticRegression`**: Importa el modelo de clasificación de Regresión Logística de Scikit-learn. Este es un algoritmo que se utiliza para problemas de clasificación binaria (en nuestro caso, predecir si un pasajero sobrevivió o no).
* **`from sklearn.metrics import accuracy_score`**: Importa la función `accuracy_score` de Scikit-learn. La exactitud es una métrica que nos dice qué porcentaje de las predicciones de nuestro modelo fueron correctas.
* **`from sklearn.impute import SimpleImputer`**: Importa la clase `SimpleImputer` de Scikit-learn. Esta herramienta nos ayudará a manejar los valores faltantes (NaN) en nuestros datos, reemplazándolos con una estrategia específica (como la media o la moda).
* **`from sklearn.preprocessing import OneHotEncoder`**: Importa la clase `OneHotEncoder` de Scikit-learn. Esta técnica se utiliza para convertir variables categóricas (textuales o con un número limitado de categorías) en un formato numérico que los algoritmos de Machine Learning pueden entender.

**2. Separación de la Variable Objetivo y las Características:**

```python
# Asumiendo que 'df' es tu DataFrame cargado en el Paso 1

# 1. Separar la variable objetivo
X = df.drop('survived', axis=1)
y = df['survived']
```

* Asumimos que `df` es el DataFrame que cargamos y analizamos en el Paso 1.
* **`X = df.drop('survived', axis=1)`**: Creamos un nuevo DataFrame llamado `X` que contiene todas las columnas del DataFrame original `df`, **excepto** la columna `'survived'`. Consideramos las columnas en `X` como las **características predictoras** o variables independientes que usaremos para predecir la variable objetivo. El argumento `axis=1` indica que queremos eliminar una columna.
* **`y = df['survived']`**: Creamos una Serie de Pandas llamada `y` que contiene **solo** la columna `'survived'` del DataFrame original. Esta es nuestra **variable objetivo** o variable dependiente, la cual queremos predecir con nuestro modelo.

**3. Identificación de Tipos de Columnas:**

```python
# 2. Identificar columnas numéricas y categóricas
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns
```

* Para aplicar diferentes estrategias de preprocesamiento a diferentes tipos de datos, primero identificamos las columnas numéricas y categóricas en nuestro conjunto de características `X`.
* **`numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns`**: Selecciona todas las columnas en `X` cuyo tipo de dato es entero (`int64`) o número de punto flotante (`float64`) y guarda sus nombres en la lista `numerical_cols`.
* **`categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns`**: Selecciona todas las columnas en `X` cuyo tipo de dato es objeto (generalmente texto), categoría o booleano, y guarda sus nombres en la lista `categorical_cols`.

**4. Imputación de Valores Faltantes:**

```python
# 3. Imputar valores faltantes
# Imputación numérica con la media
numerical_imputer = SimpleImputer(strategy='mean')
X[numerical_cols] = numerical_imputer.fit_transform(X[numerical_cols])

# Imputación categórica con la moda
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = categorical_imputer.fit_transform(X[categorical_cols])

print("DataFrame después de la imputación:")
print(X.head())
print("\nValores nulos después de la imputación:")
print(X.isnull().sum())
```

* Aquí manejamos los valores faltantes (NaN) que pudimos haber identificado en el Paso 1.
* **`numerical_imputer = SimpleImputer(strategy='mean')`**: Creamos un objeto `SimpleImputer` para las columnas numéricas. La estrategia `'mean'` indica que los valores faltantes en estas columnas serán reemplazados por la media de los valores existentes en cada columna.
* **`X[numerical_cols] = numerical_imputer.fit_transform(X[numerical_cols])`**: Primero, `fit_transform` calcula la media de cada columna numérica en `X` y luego reemplaza los valores faltantes en esas mismas columnas con sus respectivas medias. El resultado se asigna de nuevo a las columnas numéricas de `X`, modificando el DataFrame.
* **`categorical_imputer = SimpleImputer(strategy='most_frequent')`**: Creamos otro `SimpleImputer` para las columnas categóricas. La estrategia `'most_frequent'` indica que los valores faltantes en estas columnas serán reemplazados por la moda (el valor que aparece con mayor frecuencia) en cada columna.
* **`X[categorical_cols] = categorical_imputer.fit_transform(X[categorical_cols])`**: De manera similar, `fit_transform` calcula la moda de cada columna categórica en `X` y luego reemplaza los valores faltantes con sus respectivas modas. El resultado se asigna de vuelta a las columnas categóricas de `X`.
* Las siguientes líneas de `print` muestran las primeras filas del DataFrame después de la imputación y la cantidad de valores nulos por columna, lo que debería confirmar que ya no tenemos valores faltantes.

**5. Codificación de Variables Categóricas (One-Hot Encoding):**

```python
# 4. Codificar variables categóricas usando One-Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse=False para obtener un array NumPy
X_encoded = encoder.fit_transform(X[categorical_cols])
feature_names = encoder.get_feature_names_out(categorical_cols)
X_encoded_df = pd.DataFrame(X_encoded, index=X.index, columns=feature_names)

# Concatenar las columnas numéricas con las columnas codificadas
X_processed = pd.concat([X[numerical_cols], X_encoded_df], axis=1)

print("\nDataFrame después del One-Hot Encoding:")
print(X_processed.head())
print("\nForma del DataFrame preprocesado:", X_processed.shape)
```

* Aquí convertimos las variables categóricas en un formato numérico adecuado para nuestro modelo de Regresión Logística.
* **`encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)`**: Creamos un objeto `OneHotEncoder`.
    * `handle_unknown='ignore'` le dice al encoder que ignore las categorías desconocidas que puedan aparecer en el conjunto de prueba pero no en el de entrenamiento, en lugar de generar un error.
    * `sparse_output=False` hace que la salida sea un array NumPy denso en lugar de una matriz dispersa, lo cual es más fácil de trabajar en este caso.
* **`X_encoded = encoder.fit_transform(X[categorical_cols])`**: Primero, `fit` aprende las categorías únicas presentes en cada columna categórica de `X`. Luego, `transform` aplica la codificación one-hot, creando nuevas columnas binarias (0 o 1) para cada categoría única. El resultado es un array NumPy llamado `X_encoded`.
* **`feature_names = encoder.get_feature_names_out(categorical_cols)`**: Obtiene los nombres de las nuevas columnas creadas por el one-hot encoding. Estos nombres se basan en los nombres de las columnas originales y las categorías únicas dentro de ellas (por ejemplo, `sex_male`, `embarked_S`).
* **`X_encoded_df = pd.DataFrame(X_encoded, index=X.index, columns=feature_names)`**: Convertimos el array NumPy `X_encoded` en un DataFrame de Pandas llamado `X_encoded_df`, utilizando el mismo índice que el DataFrame original `X` y los nombres de las características obtenidos del encoder.
* **`X_processed = pd.concat([X[numerical_cols], X_encoded_df], axis=1)`**: Concatenamos (unimos) las columnas numéricas originales de `X` con las nuevas columnas codificadas en `X_encoded_df` a lo largo de las columnas (`axis=1`). El resultado es nuestro DataFrame preprocesado `X_processed`, que ahora contiene solo datos numéricos.
* Las siguientes líneas de `print` muestran las primeras filas y la forma del DataFrame preprocesado. El número de columnas habrá aumentado debido a la expansión de las variables categóricas.

**6. División de los Datos en Conjuntos de Entrenamiento y Prueba:**

```python
# 5. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("\nForma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)
```

* Dividimos nuestro conjunto de datos preprocesado en dos partes: un conjunto de entrenamiento para que el modelo aprenda y un conjunto de prueba para evaluar qué tan bien generaliza a datos nuevos.
* **`X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)`**: La función `train_test_split` toma las características preprocesadas (`X_processed`) y la variable objetivo (`y`) como entrada.
    * `test_size=0.2` especifica que el 20% de los datos se utilizará para el conjunto de prueba, mientras que el 80% restante se utilizará para el conjunto de entrenamiento.
    * `random_state=42` es una semilla para el generador de números aleatorios. Usar una semilla asegura que la división de los datos sea la misma cada vez que se ejecuta el código, lo que facilita la reproducibilidad de los resultados.
* Las líneas de `print` muestran la forma (número de filas y columnas) de los conjuntos de entrenamiento y prueba para las características (`X_train`, `X_test`) y la variable objetivo (`y_train`, `y_test`).

**7. Inicialización y Entrenamiento del Modelo de Regresión Logística:**

```python
# 6. Inicializar y entrenar un modelo de Regresión Logística
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
model.fit(X_train, y_train)
```

* Creamos una instancia del modelo de Regresión Logística y lo entrenamos con nuestros datos de entrenamiento preprocesados.
* **`model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)`**: Inicializamos un objeto `LogisticRegression`.
    * `solver='liblinear'` es un algoritmo de optimización adecuado para conjuntos de datos pequeños y medianos como el nuestro.
    * `random_state=42` asegura la reproducibilidad del entrenamiento del modelo.
    * `max_iter=1000` establece el número máximo de iteraciones para que el solver converja.
* **`model.fit(X_train, y_train)`**: Entrenamos el modelo utilizando el conjunto de entrenamiento (`X_train` para las características y `y_train` para la variable objetivo). Durante el entrenamiento, el modelo aprende la relación entre las características y la probabilidad de supervivencia.

**8. Realización de Predicciones en el Conjunto de Prueba:**

```python
# 7. Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
```

* Una vez que el modelo está entrenado, lo utilizamos para hacer predicciones sobre el conjunto de prueba, que el modelo nunca ha visto antes.
* **`y_pred = model.predict(X_test)`**: El método `predict` del modelo toma las características del conjunto de prueba (`X_test`) como entrada y devuelve un array (`y_pred`) que contiene las predicciones de supervivencia (0 o 1) para cada pasajero en el conjunto de prueba.

**9. Evaluación del Rendimiento del Modelo:**

```python
# 8. Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("\nExactitud del modelo en el conjunto de prueba (con imputación y codificación simplificadas):", accuracy)
```

* Finalmente, evaluamos qué tan bien se desempeñó nuestro modelo comparando sus predicciones con los valores reales de la variable objetivo en el conjunto de prueba.
* **`accuracy = accuracy_score(y_test, y_pred)`**: La función `accuracy_score` compara las etiquetas verdaderas del conjunto de prueba (`y_test`) con las predicciones de nuestro modelo (`y_pred`) y calcula la exactitud, que es la proporción de predicciones correctas.
* La línea de `print` muestra la exactitud del modelo en el conjunto de prueba, dándonos una idea de su rendimiento en datos no vistos.

---

Esta explicación detallada debería proporcionar a tus estudiantes una comprensión clara de cada paso del código, desde la importación de librerías hasta la evaluación del modelo. ¡Espero que sea útil! ¿Qué te gustaría abordar a continuación?


---

# **Explicación de los Resultados Paso 3:**

**1. DataFrame después de la Imputación:**

```
   pclass     sex   age  sibsp  parch     fare embarked  class    who  \
0     3.0    male  22.0    1.0    0.0   7.2500        S  Third    man
1     1.0  female  38.0    1.0    0.0  71.2833        C  First  woman
2     3.0  female  26.0    0.0    0.0   7.9250        S  Third  woman
3     1.0  female  35.0    1.0    0.0  53.1000        S  First  woman
4     3.0    male  35.0    0.0    0.0   8.0500        S  Third    man

  adult_male deck  embark_town alive  alone
0       True    C  Southampton    no  False
1      False    C    Cherbourg   yes  False
2      False    C  Southampton   yes   True
3      False    C  Southampton   yes  False
4       True    C  Southampton    no   True
```

* Este es un vistazo a las primeras filas del DataFrame después de que aplicamos la **imputación de valores faltantes**.
* **Columnas Numéricas (como `age`, `fare`):** Los valores `NaN` que existían previamente en estas columnas han sido reemplazados por la **media** de los valores presentes en cada una de esas columnas. Aunque no vemos un ejemplo directo en estas primeras filas (ya que no tenían `NaN`), si hubiera habido un `NaN` en la columna `age`, por ejemplo, se habría llenado con la edad promedio de todos los pasajeros.
* **Columnas Categóricas (como `embarked`, `deck`, `embark_town`):** Los valores faltantes en estas columnas han sido reemplazados por la **moda**, es decir, el valor más frecuente que aparecía en cada una de estas columnas. En la columna `deck`, vemos una 'C' en todas las primeras filas, lo que podría indicar que 'C' era la moda (el valor más común) si algunas de estas filas originalmente tenían un valor faltante en esa columna.

**2. Valores nulos después de la imputación:**

```
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64
```

* Este resultado es crucial. Muestra el **conteo de valores nulos (NaN)** para cada columna después de aplicar la imputación.
* Vemos que para cada una de las 14 columnas, el conteo de valores nulos es **0**. Esto significa que nuestra imputación ha sido exitosa y **todos los valores faltantes han sido reemplazados** utilizando las estrategias que definimos (media para numéricas, moda para categóricas).

**3. DataFrame después del One-Hot Encoding:**

```
   pclass   age  sibsp  parch     fare  sex_female  sex_male  embarked_C  \
0     3.0  22.0    1.0    0.0   7.2500         0.0       1.0         0.0
1     1.0  38.0    1.0    0.0  71.2833         1.0       0.0         1.0
2     3.0  26.0    0.0    0.0   7.9250         1.0       0.0         0.0
3     1.0  35.0    1.0    0.0  53.1000         1.0       0.0         0.0
4     3.0  35.0    0.0    0.0   8.0500         0.0       1.0         0.0

   embarked_Q  embarked_S  ...  deck_E  deck_F  deck_G  embark_town_Cherbourg  \
0         0.0         1.0  ...     0.0     0.0     0.0                    0.0
1         0.0         0.0  ...     0.0     0.0     0.0                    1.0
2         0.0         1.0  ...     0.0     0.0     0.0                    0.0
3         0.0         1.0  ...     0.0     0.0     0.0                    0.0
4         0.0         1.0  ...     0.0     0.0     0.0                    0.0

   embark_town_Queenstown  embark_town_Southampton  alive_no  alive_yes  \
0                     0.0                      1.0       1.0        0.0
1                     0.0                      0.0       0.0        1.0
2                     0.0                      1.0       0.0        1.0
3                     0.0                      1.0       0.0        1.0
4                     0.0                      1.0       1.0        0.0

   alone_False  alone_True
0          1.0         0.0
1          1.0         0.0
2          0.0         1.0
3          1.0         0.0
4          0.0         1.0

[5 rows x 32 columns]
```

* Aquí vemos el resultado de aplicar **One-Hot Encoding** a las columnas categóricas.
* Las columnas numéricas originales (`pclass`, `age`, `sibsp`, `parch`, `fare`) se mantienen intactas.
* Las columnas categóricas han sido transformadas. Por ejemplo, la columna original `sex` con valores 'male' y 'female' se ha convertido en dos nuevas columnas binarias: `sex_female` y `sex_male`. Para cada fila, una de estas columnas tendrá un valor de 1.0 (indicando la presencia de esa categoría) y la otra tendrá 0.0.
* De manera similar, la columna `embarked` (con valores como 'S', 'C', 'Q') se ha expandido en las columnas `embarked_C`, `embarked_Q`, y `embarked_S`.
* Este proceso se ha repetido para todas las columnas categóricas (`class`, `who`, `adult_male`, `deck`, `embark_town`, `alive`, `alone`), creando nuevas columnas binarias para cada categoría única dentro de ellas.
* La **forma del DataFrame preprocesado** ahora es `(891, 32)`. Originalmente teníamos 15 columnas (incluyendo la variable objetivo `survived`). Después de eliminar `survived` para el preprocesamiento y aplicar one-hot encoding a las columnas categóricas, el número de columnas ha aumentado significativamente (de 14 a 32) debido a la expansión de las variables categóricas en múltiples columnas binarias.

**4. Formas de los conjuntos de entrenamiento y prueba:**

```
Forma de X_train: (712, 32)
Forma de X_test: (179, 32)
Forma de y_train: (712,)
Forma de y_test: (179,)
```

* Después de preprocesar los datos, los dividimos en un **conjunto de entrenamiento** y un **conjunto de prueba**.
* `X_train` contiene las características preprocesadas para 712 pasajeros que se utilizarán para entrenar nuestro modelo. Tiene 32 columnas, correspondientes a las características numéricas originales y las nuevas columnas creadas por el one-hot encoding.
* `X_test` contiene las mismas 32 características preprocesadas para 179 pasajeros que se utilizarán para evaluar el rendimiento de nuestro modelo entrenado.
* `y_train` contiene la variable objetivo (`survived`) para los 712 pasajeros del conjunto de entrenamiento. Tiene una sola dimensión.
* `y_test` contiene la variable objetivo para los 179 pasajeros del conjunto de prueba, que usaremos para comparar con las predicciones de nuestro modelo.

**5. Exactitud del modelo en el conjunto de prueba:**

```
Exactitud del modelo en el conjunto de prueba (con imputación y codificación simplificadas): 1.0
```

* Este es el resultado de la **evaluación de nuestro modelo de Regresión Logística** en el conjunto de prueba.
* Una exactitud de **1.0** (o 100%) significa que **nuestro modelo predijo correctamente la supervivencia de todos los pasajeros en el conjunto de prueba**.

**Puntos Importantes :**

* Hemos logrado **manejar los valores faltantes** utilizando estrategias básicas de imputación.
* Hemos convertido las **variables categóricas en un formato numérico** que el modelo de Machine Learning puede entender a través del One-Hot Encoding.
* El número de características (columnas) ha aumentado debido al One-Hot Encoding.
* Hemos dividido los datos preprocesados para entrenar y evaluar el modelo.
* El modelo de Regresión Logística, con este preprocesamiento, ha alcanzado una exactitud perfecta en el conjunto de prueba.

**Precaución sobre la Exactitud del 100%:**

Es importante mencionar que una exactitud del 100% en un conjunto de prueba **debe ser examinada con escepticismo**, especialmente en problemas del mundo real. Podría indicar:

* **Un conjunto de prueba demasiado pequeño:** Con pocos ejemplos, es más fácil obtener una predicción perfecta por casualidad.
* **Fugas de información (data leakage):** Podría haber alguna información en las características que indirectamente revela la variable objetivo.
* **Un problema demasiado simple para el modelo:** En algunos casos raros, el problema podría ser inherentemente muy fácil de predecir.

En los siguientes pasos, podríamos explorar estas posibilidades y aprender sobre la importancia de validar los modelos de manera robusta.



# **¿Por Qué Usamos One-Hot Encoding en Lugar de Label Encoding para Variables Categóricas?**

La elección entre One-Hot Encoding y Label Encoding depende fundamentalmente de la **naturaleza de la variable categórica** que estamos codificando, específicamente si existe o no una **relación de orden inherente** entre sus categorías.

**Label Encoding (Codificación de Etiquetas):**

* **¿Qué hace?** Asigna un número entero único a cada categoría en la variable. Por ejemplo, si la columna 'Color' tiene las categorías 'Rojo', 'Verde' y 'Azul', Label Encoding podría convertirlas en 0, 1 y 2 respectivamente.
* **¿Cuándo podría ser apropiado?** Label Encoding es más apropiado para **variables categóricas ordinales**. Una variable es ordinal si sus categorías tienen un orden o una jerarquía significativa. Ejemplos:
    * 'Tamaño': 'Pequeño', 'Mediano', 'Grande' (existe un orden)
    * 'Nivel de Educación': 'Primaria', 'Secundaria', 'Universidad' (existe un orden)
    * 'Satisfacción': 'Bajo', 'Medio', 'Alto' (existe un orden)
* **¿Cuál es el problema con variables nominales?** El principal problema de aplicar Label Encoding a **variables categóricas nominales** (donde no existe un orden inherente entre las categorías, como 'Color', 'Ciudad', 'Sexo', 'Puerto de Embarque') es que **introduce una noción de orden o jerarquía que no existe en realidad**.

    * Para nuestro ejemplo de 'Color' (Rojo: 0, Verde: 1, Azul: 2), el modelo podría interpretar erróneamente que 'Verde' es "mayor que" 'Rojo' y que 'Azul' es "mayor que" ambos. Esto puede confundir al algoritmo y llevar a un rendimiento subóptimo, ya que el modelo podría asignar más peso o importancia a ciertas categorías basándose en el valor numérico asignado, en lugar de en su verdadera relación con la variable objetivo.

**One-Hot Encoding (Codificación "Uno Caliente"):**

* **¿Qué hace?** Crea nuevas columnas binarias (con valores 0 o 1) para cada categoría única en la variable original. Por ejemplo, la columna 'Color' ('Rojo', 'Verde', 'Azul') se convertiría en tres nuevas columnas: 'Color_Rojo', 'Color_Verde' y 'Color_Azul'. Para cada fila, solo una de estas columnas tendrá un valor de 1 (indicando que esa era la categoría original), mientras que las otras tendrán 0.
* **¿Cuándo es apropiado?** One-Hot Encoding es la técnica preferida para **variables categóricas nominales**. Al crear columnas separadas para cada categoría, evitamos introducir cualquier tipo de relación de orden o magnitud entre ellas. Cada categoría se trata como independiente de las demás.
* **¿Por qué es mejor para variables nominales?**
    * **Evita la falsa relación de orden:** Cada categoría tiene su propia columna, por lo que el modelo no asume ninguna jerarquía entre ellas.
    * **Permite al modelo aprender relaciones específicas:** El modelo puede aprender una relación diferente entre cada categoría y la variable objetivo. Por ejemplo, la probabilidad de supervivencia podría ser diferente para hombres y mujeres, y el One-Hot Encoding permite al modelo capturar estas diferencias individualmente a través de las columnas 'sex_male' y 'sex_female'.

**En el Contexto del Dataset del Titanic:**

La mayoría de las variables categóricas en el dataset del Titanic son **nominales**. Considera ejemplos como:

* **`sex` (male/female):** No hay un orden inherente entre ser hombre o mujer en términos de supervivencia.
* **`embarked` (C, Q, S):** Los diferentes puertos de embarque no tienen una relación ordinal natural.
* **`class` (First, Second, Third):** Aunque existe un orden socioeconómico, al tratarlos directamente como categorías nominales a través de One-Hot Encoding, permitimos que el modelo aprenda la relación específica de cada clase con la supervivencia sin imponer un gradiente lineal. (Podría haber argumentos para Label Encoding aquí si se quisiera enfatizar el orden, pero One-Hot es más común para evitar suposiciones).
* **Otras como `who`, `deck`, `embark_town`, `alive`, `alone` también son principalmente nominales.**

Al aplicar Label Encoding a estas variables nominales, estaríamos implícitamente diciendo al modelo que existe una "mayor que" o "menor que" entre las categorías, lo cual no es cierto y podría distorsionar el aprendizaje del modelo.

**Excepción Potencial:**

Podría haber algunas variables categóricas en el dataset (como `class` mencionado anteriormente o quizás alguna codificación de rangos de edad si se hubiera hecho como categorías) donde un Label Encoding *podría* ser considerado si el orden es realmente significativo y se quiere que el modelo lo interprete como tal. Sin embargo, incluso en esos casos, One-Hot Encoding suele ser más seguro para evitar interpretaciones erróneas por parte del modelo.

**En resumen, usamos One-Hot Encoding para las variables categóricas nominales en el dataset del Titanic (y en muchos otros problemas de clasificación) porque:**

* **Preserva la independencia de las categorías.**
* **Evita introducir relaciones de orden o magnitud artificiales.**
* **Permite que el modelo aprenda la influencia específica de cada categoría en la variable objetivo.**

Además de los conceptos, siempre se debe enfatizar la importancia de **entender la naturaleza de los datos** antes de aplicar cualquier técnica de codificación.